                        # Lending Club Exploratory Data Analysis Case Study
                            By Gopinath Moganasundaram and Mir Riaz Ahmed

## Introduction

### Problem Statement
The data set given (loan.csv) contains information about past loan applicants and whether they ‘defaulted’ or not. The aim is to analyze the data and identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.
 
In this case study, we will use EDA to understand how consumer attributes and loan attributes influence the tendency of default.

### Approach
The approach is to load the data, clean it up, perform EDA concepts on the data and then derive conclusion/recommendations on the driving factors behind loan defaults

## Data Loading

In [26]:
# Import needed libraries
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [27]:
# Load the data
loan_df = pd.read_csv('loan.csv')
loan_df.shape

(39717, 111)

In [28]:
# Check top 5 rows to understand columns and the kind of data that is present
loan_df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


## Data Cleaning

### Removing null columns

##### Remove columns which have all nulls
Using shape, we found that there are total 39717 rows.
Checking nulls shows that there are a lot of columns who have nulls in all the rows.
We can remove those columns from the dataframe

In [29]:
# Get 
loan_df.isnull().sum().sort_values().tail(10)

total_rev_hi_lim              39717
inq_fi                        39717
total_cu_tl                   39717
inq_last_12m                  39717
acc_open_past_24mths          39717
avg_cur_bal                   39717
bc_open_to_buy                39717
bc_util                       39717
mo_sin_old_rev_tl_op          39717
total_il_high_credit_limit    39717
dtype: int64

In [30]:
# Drop null columns and then check the shape
loan_df = loan_df.dropna(axis=1, how='all')
loan_df.shape

(39717, 57)

#### Remove columns which have more than 40% of nulls
Now that we are left with only 57 columns, lets check the null percentage of columns and remove columns with more than 40% nulls

In [31]:
# checking mean on isnull for all columns
loan_df.isnull().mean().sort_values().tail(10)

collections_12_mths_ex_med    0.001410
chargeoff_within_12_mths      0.001410
last_pymnt_d                  0.001788
pub_rec_bankruptcies          0.017549
emp_length                    0.027066
emp_title                     0.061913
desc                          0.325805
mths_since_last_delinq        0.646625
mths_since_last_record        0.929854
next_pymnt_d                  0.971297
dtype: float64

In [32]:
# Removing columns mths_since_last_delinq, mths_since_last_record, next_pymnt_d
loan_df = loan_df.drop(['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d'], axis=1)
loan_df.shape

(39717, 54)

In [75]:
# checking mean on isna for all column
loan_df.isna().mean().sort_values().tail(10)*100

member_id                0.000000
home_ownership           0.000000
last_credit_pull_d       0.005036
title                    0.027696
revol_util               0.125891
last_pymnt_d             0.178765
pub_rec_bankruptcies     1.754916
emp_length               2.706650
emp_title                6.191303
desc                    32.580507
dtype: float64

In [76]:
# Removing column 'desc'
loan_df = loan_df.drop('desc', axis=1)
loan_df.shape

(39717, 44)

#### Remove columns which have same values for all rows
Let's figure out those columns with nunique and remove them

In [35]:
loan_df.nunique().sort_values().head(10)

tax_liens                     1
delinq_amnt                   1
chargeoff_within_12_mths      1
acc_now_delinq                1
application_type              1
policy_code                   1
initial_list_status           1
collections_12_mths_ex_med    1
pymnt_plan                    1
term                          2
dtype: int64

In [42]:
loan_df = loan_df.drop(['tax_liens', 'delinq_amnt', 'chargeoff_within_12_mths', 'acc_now_delinq', 'application_type'], axis=1)
loan_df = loan_df.drop(['policy_code', 'initial_list_status', 'collections_12_mths_ex_med', 'pymnt_plan'], axis=1)
loan_df.shape

(39717, 45)

#### Remove irrelevant columns
Now let's figure out columns that are not relevant to our analysis and remove them

In [44]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')

In [58]:
# The id columns are not needed for analysis
irrelevant_columns = ['id', 'member_id']

# Other irrelevant columns
irrelevant_columns.append('url')
irrelevant_columns.append('title')
irrelevant_columns

['id', 'member_id', 'desc', 'title']

In [72]:
loan_df['dti'].value_counts()
loan_df.shape

loan_df.isnull().mean().sort_values().tail(10)
loan_df.isna().mean().sort_values().tail(10)

member_id               0.000000
home_ownership          0.000000
last_credit_pull_d      0.000050
title                   0.000277
revol_util              0.001259
last_pymnt_d            0.001788
pub_rec_bankruptcies    0.017549
emp_length              0.027066
emp_title               0.061913
desc                    0.325805
dtype: float64

#### Remove Duplicate Rows

In [68]:
# Check if there are any duplicate rows and remove them
print("Duplicate rows: ", loan_df[loan_df.duplicated()].shape)
# No duplicate rows found

Duplicate rows:  (0, 45)


#### Remove records where status = 'Current'

In [ ]:
loan_df = loan_df[loan_df.loan_status != "Current"]
loan_df.loan_status.unique()

#### Correct the data type of the columns

In [ ]:
loan_data.revol_util = pd.to_numeric(loan_data.revol_util.apply(lambda x : x.split('%')[0]))
loan_data.int_rate = pd.to_numeric(loan_data.int_rate.apply(lambda x : x.split('%')[0]))
loan_data.emp_length = pd.to_numeric(loan_data.emp_length.apply(lambda x: 0 if "<" in x else (x.split('+')[0] if "+" in x else x.split()[0])))
loan_data.head()

#### Removing outliers

In [ ]:
sns.boxplot(loan_data['annual_inc'])
-----
quantile_info = loan_data.annual_inc.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info
-----
per_95_annual_inc = loan_data['annual_inc'].quantile(0.95)
loan_data = loan_data[loan_data.annual_inc <= per_95_annual_inc]
------
sns.boxplot(loan_data.annual_inc)
-------
sns.boxplot(loan_data.dti) # no outlier
-------
sns.boxplot(loan_data.loan_amnt)
-------
loan_data.loan_amnt.quantile([0.75,0.90,0.95,0.97,0.975, 0.98, 0.99, 1.0])
-------
sns.boxplot(loan_data.funded_amnt_inv)
-------
loan_data.funded_amnt_inv.quantile([0.5,0.75,0.90,0.95,0.97,0.975, 0.98,0.985, 0.99, 1.0])
-------

#### Visualization


In [ ]:
sns.countplot(x = 'loan_status', data = loan_data)
-------
loan_data.sub_grade = pd.to_numeric(loan_data.sub_grade.apply(lambda x : x[-1]))
loan_data.sub_grade.head()
-------
fig, ax = plt.subplots(figsize=(12,7))
sns.set_palette('colorblind')
sns.countplot(x = 'grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] , hue = 'sub_grade',data = loan_data[loan_data.loan_status == 'Charged Off'])
-------
sns.countplot(x = 'grade', data = loan_data[loan_data.loan_status == 'Charged Off'], order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
-------
# Analyze home ownership
#checking unique values for home_ownership
loan_data['home_ownership'].unique()
--------
# There are only 3 values with 'NONE' so replacing with OTHER
#replacing 'NONE' with 'OTHERS'
loan_data['home_ownership'].replace(to_replace = ['NONE'],value='OTHER',inplace = True)
--------
#checking unique values for home_ownership again
loan_data['home_ownership'].unique()
---------
fig, ax = plt.subplots(figsize = (6,4))
ax.set(yscale = 'log')
sns.countplot(x='home_ownership', data=loan_data[loan_data['loan_status']=='Charged Off'])
---------
# Analyzing purpose
fig, ax = plt.subplots(figsize = (12,8))
ax.set(xscale = 'log')
sns.countplot(y ='purpose', data=loan_data[loan_data.loan_status == 'Charged Off'])
---------
# Creating bins for numerical rows
#creating bins for int_rate,open_acc,revol_util,total_acc
loan_data['int_rate_groups'] = pd.cut(loan_data['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])
loan_data['open_acc_groups'] = pd.cut(loan_data['open_acc'],bins = 5,precision =0,labels=['2-10','10-19','19-27','27-36','36-44'])
loan_data['revol_util_groups'] = pd.cut(loan_data['revol_util'], bins=5,precision =0,labels=['0-20','20-40','40-60','60-80','80-100'])
loan_data['total_acc_groups'] = pd.cut(loan_data['total_acc'], bins=5,precision =0,labels=['2-20','20-37','37-55','55-74','74-90'])
loan_data['annual_inc_groups'] = pd.cut(loan_data['annual_inc'], bins=5,precision =0,labels =['3k-31k','31k-58k','58k-85k','85k-112k','112k-140k'])
---------
    # Viewing new bins created
    loan_data.head()
---------
# Analyzing interest rate wrt to interest rate bins
fig, ax = plt.subplots(figsize = (15,10))
plt.subplot(221)
sns.countplot(x='int_rate_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
plt.xlabel('Interest Rate')
plt.subplot(222)
sns.countplot(x='emp_length', data=loan_data[loan_data.loan_status == 'Charged Off'])
----------
#### Similarly analyzing open_acc,revol_util,total_acc,annual_inc
fig, ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='open_acc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
----------
sns.countplot(x='revol_util_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
---------
fig, ax = plt.subplots(figsize = (8,6))
ax.set_yscale('log')
sns.countplot(x='total_acc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
----------
fig, ax = plt.subplots(figsize = (10,6))
sns.countplot(x='annual_inc_groups', data=loan_data[loan_data.loan_status == 'Charged Off'])
---------
sns.countplot(y='term', data=loan_data[loan_data['loan_status']=='Charged Off'])
---------
sns.countplot(x='verification_status', data=loan_data[loan_data['loan_status']=='Charged Off'])
---------
fig,ax = plt.subplots(figsize = (10,8))
ax.set_yscale('log')
sns.countplot(x='inq_last_6mths', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------
fig,ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='pub_rec', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------
# Analyzing issue month and year
## Extracting month and year
df_month_year = loan_data['issue_d'].str.partition("-", True)     
loan_data['issue_month']=df_month_year[0]                       
loan_data['issue_year']='20' + df_month_year[2]
----------
loan_data.head()
----------
plt.figure(figsize=(15,15))
plt.subplot(221)
sns.countplot(x='issue_month', data=loan_data[loan_data['loan_status']=='Charged Off'])
plt.subplot(222)
sns.countplot(x='issue_year', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------

In [ ]:
### Maximum number of defaults occured when the loan was sanctioned/issued in Dec. Loan issued in the year 2011 were also as compared to other years

### Analyzing installment,dti, loan_amnt 
-----------
loan_data['installment_groups'] = pd.cut(loan_data['installment'], bins=10,precision =0,labels=['14-145','145-274','274-403','403-531','531-660','660-789','789-918','918-1047','1047-1176','1176-1305'])
loan_data['funded_amnt_inv_group'] = pd.cut(loan_data['funded_amnt_inv'], bins=7,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k']) ## bin is starting from -35?
loan_data['loan_amnt_groups'] = pd.cut(loan_data['loan_amnt'], bins=7,precision =0,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_data['dti_groups'] = pd.cut(loan_data['dti'], bins=5,precision =0,labels=['0-6','6-12','12-18','18-24','24-30'])
----------
fig,ax = plt.subplots(figsize = (12,5))
ax.set_yscale('log')
sns.countplot(x='funded_amnt_inv_group', data=loan_data[loan_data['loan_status']=='Charged Off'])
---------
fig,ax = plt.subplots(figsize = (12,5))
ax.set_yscale('log')
sns.countplot(x='funded_amnt_inv_group', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------
sns.countplot(x='dti_groups', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='installment_groups', data=loan_data[loan_data['loan_status']=='Charged Off'])
----------


In [ ]:
# Observations

### The above analysis with respect to the charged off loans for each variable suggests the following. There is a more probability of defaulting when : 

- Applicants having house_ownership as 'RENT'
- Applicants who use the loan to clear other debts
- Applicants who receive interest at the rate of 13-17%
- Applicants who have an income of range 31201 - 58402
- Applicants who have 20-37 open_acc
- Applicants with employement length of 10
- When funded amount by investor is between 5000-10000
- Loan amount is between 5429 - 10357
- Dti is between 12-18
- When monthly installments are between 145-274
- Term of 36 months
- When the loan status is Not verified
- When the no of enquiries in last 6 months is 0
- When the number of derogatory public records is 0
- When the purpose is 'debt_consolidation'
- Grade is 'B'
- And a total grade of 'B5' level.
-----------------------------------------------------
### Also there is a very interesting observation from the date issued. The late months of an year indicated the high possibility of defaulting. 
- The high number of loan defaults in 2011 could be due to the financial crisis in USA (Assuming the data is of US origin)
--------------------------------------
## Analysing annual income with other columns for more insights 
------------
Annucal income vs loan purpose
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='annual_inc', y='purpose', hue ='loan_status',palette="deep")
plt.show()
-------------



In [ ]:
### Though the number of loans applied and defaulted are the highest in number for "debt_consolation", the annual income of those who applied isn't the highest. 
- Applicants with higher salary mostly applied loans for "home_improvment", "house", "renewable_energy" and "small_businesses"
#### 2.Annual income vs home ownership
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='home_ownership', y='annual_inc', hue ='loan_status',palette="pastel")
plt.show()
##### Annual income vs loan amout
sns.barplot(x = "annual_inc_groups", y = "loan_amnt", hue = 'loan_status', data = loan_data)
#### Annaul Income vs Interest rate 
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='int_rate_groups', y='annual_inc', hue ='loan_status',palette="pastel")
plt.show()
-----------------
#### Loan vs Loan purpose
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='purpose', hue ='loan_status',palette="pastel")
plt.show()
#### Loan vs house ownership
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='home_ownership', hue ='loan_status',palette="pastel")
plt.show()
#### Loan amount vs month issues and year issued
plt.figure(figsize=(20,20))
plt.subplot(221)
sns.lineplot(data =loan_data,y='loan_amnt', x='issue_month', hue ='loan_status',palette="pastel")
plt.subplot(222)
sns.lineplot(data =loan_data,y='loan_amnt', x='issue_year', hue ='loan_status',palette="pastel")
#### Loan amount vs grade
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='loan_amnt', y='grade', hue ='loan_status',palette="pastel", order=['A','B','C','D','E','F','G'])
plt.show()
-----------------------
plt.figure(figsize=(20,20))
plt.subplot(221)
sns.barplot(data =loan_data,y='loan_amnt', x='emp_length', hue ='loan_status',palette="pastel")
plt.subplot(222)
sns.barplot(data =loan_data,y='loan_amnt', x='verification_status', hue ='loan_status',palette="pastel")


In [ ]:
### Employees with longer working history got the loan approved for a higher amount. 
- Looking at the verification status data, verified loan applications tend to have higher loan amount. Which might indicate that the firms are first verifying the loans with higher values.
------
# Grade vs Interest rate
plt.figure(figsize=(10,10))
sns.barplot(data =loan_data,x='int_rate', y='grade', hue ='loan_status',palette="pastel", order=['A','B','C','D','E','F','G'])
plt.show()
-----------------
# fig,ax = plt.subplots(figsize = (15,6))
plt.tight_layout()
sns.catplot(data =loan_data,y ='int_rate', x ='loan_amnt_groups', hue ='loan_status',palette="pastel",kind = 'box')
--------------

In [ ]:
### The interest rate for charged off loans is pretty high than that of fully paid loans in all the loan_amount groups. 
# - This can be a pretty strong driving factor for loan defaulting.
sns.catplot(x = 'term', y = 'loan_amnt', data = loan_data,hue = 'loan_status', kind = 'bar')


In [ ]:
### Applicants who applied and defaulted have no significant difference in loan_amounts.
- Which means that applicants applying for long term has applied for more loan. 

# Observations

### The above analysis with respect to the charged off loans. There is a more probability of defaulting when : 

- Applicants taking loan for 'home improvement' and have income of 60k -70k
- Applicants whose home ownership is 'MORTGAGE and have income of 60-70k
- Applicants who receive interest at the rate of 21-24% and have an income of 70k-80k
- Applicants who have taken a loan in the range 30k - 35k and are charged interest rate of 15-17.5 %
- Applicants who have taken a loan for small business and the loan amount is greater than 14k
- Applicants whose home ownership is 'MORTGAGE and have loan of 14-16k
- When grade is F and loan amount is between 15k-20k
- When employment length is 10yrs and loan amount is 12k-14k 
- When the loan is verified and loan amount is above 16k
- For grade G and interest rate above 20%